# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [1]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import Reader
from firefly_api.particlegroup import ParticleGroup
import numpy as np
import pandas as pd

### Set the defaults and create the .json files

In [4]:
#create tje reader
my_reader = Reader(
    ## the name of the sub-directory that will contain your JSON files,
    ##  if you are not running python from /path/to/Firefly/data it should be the absolute path
    JSONdir = '/Users/agurvich/research/repos/Firefly/data/MyReader', 
    ## options object, if you have one you can pass it, None will generate default options, see
    ##  below for options arguments/capabilities
    options = None, 
    ## whether a startup.json file should be written, defaults to 'append'
    write_startup = True,
    max_npart_per_file= 10**4, 
    prefix = 'MyData',
    clean_JSONdir = False,
    doValidate = False
)


## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T
print(coords.shape)

#define a particle group
my_parts = ParticleGroup(
    'part',
    coords,
    ## below here I pass a few option_kwargs to set the size and color of the points
    sizeMult=5, ## make the particles a bit easier to see since there's so few of them
    color = [0,0,1,1], ## make them blue, I like blue. colors should be RGBA lists
    )

## tell the reader to keep track of this particle group
my_reader.addParticleGroup(my_parts)

## Store this info in dicts with the correct format for Firefly
outDict = {"parts":{}, "options":None}
for pg in my_reader.particleGroups:
    outDict['parts'][pg.UIname] = pg.outputToDict(np.arange(1000))
outDict['options'] = my_reader.options.outputToDict()


(8000, 3)


In [5]:
print(outDict)

{'parts': {'part': {'Coordinates': array([[-10.        , -10.        , -10.        ],
       [-10.        , -10.        ,  -8.94736842],
       [-10.        , -10.        ,  -7.89473684],
       ...,
       [ -0.52631579,  -7.89473684,   7.89473684],
       [ -0.52631579,  -7.89473684,   8.94736842],
       [ -0.52631579,  -7.89473684,  10.        ]]), 'filterKeys': array([], dtype=float64), 'colormapKeys': array([], dtype=float64), 'doSPHrad': [0]}}, 'options': {'title': 'Firefly', 'loaded': True, 'UI': True, 'UIfullscreen': True, 'UIsnapshot': True, 'UIreset': True, 'UIsavePreset': True, 'UIloadNewData': True, 'UIcameraControls': True, 'UIdecimation': True, 'UIparticle': {'part': True}, 'UIdropdown': {'part': True}, 'UIcolorPicker': {'part': True}, 'center': array([0., 0., 0.]), 'camera': None, 'cameraRotation': None, 'maxVrange': 2000.0, 'startFly': False, 'friction': None, 'stereo': False, 'stereoSep': None, 'decimate': None, 'plotNmax': {'part': None}, 'showVel': {'part': False}, 

## Send this data to the Flask app

In [7]:
import sys
import json
import requests
from IPython.display import IFrame

*This sends the data.*

In [8]:
s = pd.Series(outDict).to_json()
requests.post("http://localhost:5000/data_input", json=s)

<Response [200]>

*This shows the viewer.*

In [104]:
url = "http://localhost:5000/combined"
IFrame(url, width=700, height=300)